In [ ]:
filtered_restaurants = df.drop_duplicates().merge(filtered_restaurants[['restaurantName', 'region', 'latitude', 'longitude']].drop_duplicates(), on='restaurantName', how='left')

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import engine

# Function to extract unique values from a column of lists
def extract_unique_values(column_name):
    '''
    Function that, given a column of lists of strings, finds the unique strings among all lists.
    Input:
    column_name: dataframe column or series
    Output:
    unique_values: sorted list of unique strings present in the lists of the column
    '''
    # Initialize a set to store unique values
    unique_values = set()

    # Iterate through the column, drop NaN and duplicates first
    for lis in column_name.dropna().drop_duplicates():
      lis = eval(lis) # evaluate the string expression that evaluates to a list of strings
      if isinstance(lis, list):  # ensure the value is a list
        for el in lis:
          if isinstance(el, str):  # ensure the element is a string
            unique_values.add(el)  # add the element to the set

    # Return a sorted list of unique values
    return sorted(unique_values)

# Get unique values for credit cards and facilities
unique_credit_cards = extract_unique_values(filtered_restaurants.creditCards)
unique_services_facilities = extract_unique_values(filtered_restaurants.facilitiesServices)

# Saved regions in alphabetical order
regions = filtered_restaurants.region.dropna().drop_duplicates().sort_values()

# Text input widgets for restaurant name, city, cuisine type
restaurant_name = widgets.Text(placeholder='Type restaurant name', description='Restaurant:') # text input for restaurant name
city = widgets.Text(placeholder='Type city', description='City:') # text input for city
cuisine_type = widgets.Text(placeholder='Type cuisine type', description='Cuisine:') # text input for cuisine type

# Dropdown widgets for price range, region
price_range = widgets.Dropdown(options=['€', '€€', '€€€', '€€€€'], value='€€', description='Price Range:') # dropdown choice for price range
region = widgets.Dropdown(options=regions, description='Region:') # dropdown choice for regions

# Define checkboxes for credit cards
credit_card_options = unique_credit_cards
credit_card_checkboxes = [widgets.Checkbox(value=False, description=option) for option in credit_card_options]
credit_card_box = widgets.VBox(credit_card_checkboxes)

# Define checkboxes for services and facilities
facility_options = unique_services_facilities
facility_checkboxes = [widgets.Checkbox(value=False, description=option) for option in facility_options]
facility_box = widgets.VBox(facility_checkboxes)

# Display widgets
display(restaurant_name, city, cuisine_type, price_range, region)
display(widgets.Label("Credit Cards:"))
display(credit_card_box)
display(widgets.Label("Additions:"))
display(facility_box)

# Search button and output area
search_button = widgets.Button(description="Search")
output = widgets.Output()

# Define the search function to handle button click
def search_restaurants(restaurant_name, city, cuisine_type, price_range, region, credit_card_checkboxes, facility_checkboxes, output):
    with output:
        clear_output()  # clear any outputn from previous button presses

        # Gather selected options from checkboxes
        selected_credit_cards = [ccc.description for ccc in credit_card_checkboxes if ccc.value] # list of selected checkboxes for credit card (True or False)
        selected_facilities = [fc.description for fc in facility_checkboxes if fc.value]  # list of selected checkboxes for services and facilities (True or False)

        # Create search criteria dictionary
        search_criteria = {
            "restaurant_name": restaurant_name.value,
            "city": city.value,
            "cuisine_type": cuisine_type.value,
            "price_range": price_range.value,
            "region": region.value,
            "credit_cards": selected_credit_cards,
            "services_facilities": selected_facilities,
        }

        # Custom action with search criteria
        print("Search Criteria:")
        for key, value in search_criteria.items():
            print(f"{key}: {value}")

        # Insert additional logic or function calls here to handle the search
        # e.g., fetch results from a database, make an API call, etc.

# Attach the callback function to the button click using a lambda
search_button.on_click(lambda b: search_restaurants(
    restaurant_name, city, cuisine_type, price_range, region,
    credit_card_checkboxes, facility_checkboxes, output
))

# Display the button and output area
display(search_button, output)


In [ ]:
# Initialize sets of restaurants that satisfy the necessary conditions
# All restaurants in the returned search results need to satisfy these conditions
price_range_correct = set()
region_correct = set()
credit_card_correct = set()
facilities_correct = set()
n = len(filtered_restaurants)

# Find all the restaurants that satisfy the conditions
for row in range(n):
  if filtered_restaurants.iloc[row]['priceRange'] == price_range.value or not price_range.value: # correct price range
    price_range_correct.add(row)
  if filtered_restaurants.iloc[row]['region'] == region.value or not region.value: # correct region
    region_correct.add(row)
  if set(eval(filtered_restaurants.iloc[row]['creditCards'])).issubset(set(credit_card_checkboxes)) or not credit_card_checkboxes: # correct credit cards
    credit_card_correct.add(row)
  if set(eval(filtered_restaurants.iloc[row]['facilitiesServices'])).issubset(set(facility_checkboxes)) or not facility_checkboxes: # correct facilities
    facilities_correct.add(row)

'''
print(price_range_correct)
print(region_correct)
print(credit_card_correct)
print(facilities_correct)
'''

# Find the intersection of all restaurants (rows in the dataset)
restaurant_intersection = list(price_range_correct.intersection(region_correct).intersection(credit_card_correct).intersection(facilities_correct))

#print(restaurant_intersection)

# Apply cosine similarity to input restaurant name, cities, cuisine
top_names = top_k_cosine_similarity(df.iloc[restaurant_intersection], query=restaurant_name.value, k=n)
top_cities = top_k_cosine_similarity(df.iloc[restaurant_intersection], query=city.value, k=n)
top_cuisines = top_k_cosine_similarity(df.iloc[restaurant_intersection], query=cuisine_type.value, k=n)

# Make dictionaries for intersection
top_names_dict = {top_names[i][0]: top_names[i][1] for i in range(len(top_names))}
top_cities_dict = {top_cities[i][0]: top_cities[i][1] for i in range(len(top_cities))}
top_cuisines_dict = {top_cuisines[i][0]: top_cuisines[i][1] for i in range(len(top_cuisines))}

# Initialize a defaultdict to store the cumulative scores for each restaurant
cumulative_scores = defaultdict(float)

# Add scores from top_names_dict
for restaurant, score in top_names_dict.items():
    cumulative_scores[restaurant] += score

# Add scores from top_cities_dict
for restaurant, score in top_cities_dict.items():
    cumulative_scores[restaurant] += score

# Add scores from top_cuisines_dict
for restaurant, score in top_cuisines_dict.items():
    cumulative_scores[restaurant] += score

# Convert cumulative_scores back to a regular dictionary (optional)
unified_scores = dict(cumulative_scores)

# Normalize the scores dividing them all by 3
for restaurant, score in unified_scores.items():
    unified_scores[restaurant] = score / 3

unified_scores = [(restaurant, score) for restaurant, score in unified_scores.items()]

# Sort the unified scores in descending order
k = 5
if unified_scores:
 unified_scores = sorted(unified_scores, key=lambda x: x[1], reverse=True)[:k]

build_search_results(unified_scores, filtered_restaurants, ['restaurantName', 'address', 'cuisineType', 'priceRange', 'website'])